In [1]:
from dotenv import load_dotenv
load_dotenv()

True

**대화형 프롬프트에 주입하고 싶을 때는 ChatPromptTemplate을 사용합니다.**

In [2]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name} 입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
)
messages = chat_template.format_messages(
    name="테디", user_input="당신의 이름은 무엇입니까?"
)
messages

[SystemMessage(content='당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 테디 입니다.'),
 HumanMessage(content='반가워요!'),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?'),
 HumanMessage(content='당신의 이름은 무엇입니까?')]

In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    model_name="gpt-3.5-turbo",  # 모델명
    max_tokens = 50
)

chain = chat_template | model 

output = chain.invoke({'name' : "테디", 'user_input' : "당신의 이름은 무엇입니까?"})

output



#messages를 활용하고 싶으면
#model.invoke(messages).content

AIMessage(content='제 이름은 테디입니다. 어떻게 도와드릴까요?', response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 90, 'total_tokens': 114}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-beef9347-a631-426a-8f4c-366dd4f85a1e-0', usage_metadata={'input_tokens': 90, 'output_tokens': 24, 'total_tokens': 114})

In [5]:
output.content

'제 이름은 테디입니다. 어떻게 도와드릴까요?'

**MessagesPlaceholder를 사용하면 대화를 변수처럼 삽입할 수 있다.**

In [6]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.",
        ),
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다."),
    ]
)
formatted_chat_prompt = chat_prompt.format(
    word_count=5,
    conversation=[
        ("human", "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다."),
        ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
    ],
)

formatted_chat_prompt

'System: 당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.\nHuman: 안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.\nAI: 반가워요! 앞으로 잘 부탁 드립니다.\nHuman: 지금까지의 대화를 5 단어로 요약합니다.'